# Transformers 를 이용해 Backbone 사용

## Transformers 설치
- `pip install transformers`

### Tokenizer, Model Loading
- Huggingface 모델 허브에서 제공하는 처리 모델을 다운받아 로딩한다.
- 다운로드된 모델은 `사용자 home 디렉토리\.cache\huggingface` 에 저장된다.
- 미리 학습된 언어 모델을 다운받아 사용할 때는 그 언어모델이 사용한 tokenizer를 같이 받아서 사용한다.

### [Auto Classes](https://huggingface.co/docs/transformers/model_doc/auto)
- Huggingface 에서 제공하는 다양한 모델들은 손쉽게 불러오고 사용할 수 있도록 설계된 유틸리티 클래스들을 말한다.
- 미리 학습된 특정 모델의 이름(모델 허브상에서 제공되는 이름)이나 저장된 local 경로를 제공하면 해당 모델에 맞는 적절한 클래스와 구성 요소를 자동으로 로드한다.
- 사용자는 모델을 사용하기 위한 정확한 클래스를 몰라도 쉽게 다양한 종류의 모델을 사용할 수있다.

#### 주요 Auto Class
- 기본 모델 Loading
    1. **AutoModel**
       - 주어진 모델 이름에 맞는 사전 학습된 모델을 자동으로 로드한다.
       - 예: `AutoModel.from_pretrained("bert-base-uncased")`: BERT 모델을 로드한다.
    2. **AutoTokenizer**
       - 해당 모델에 적합한 토크나이저를 자동으로 로드한다.
       - 예: `AutoTokenizer.from_pretrained("bert-base-uncased")`: BERT 모델에 맞는 토크나이저를 로드한다.
    3. **AutoConfig**
       - 모델의 설정(config)을 자동으로 로드한다. 모델 설정에는 모델의 하이퍼파라미터와 모델 구조 정보가 포함된다. 이 설정을 이용해 모델을 생성할 수있다.
       - 예: `AutoConfig.from_pretrained("bert-base-uncased")`
- Task 처리 모델 Loading
    - Pretrained backbone 모델에 각 task 에 맞는 estimator layer를 추가한 모델을 생성해 제공한다.
    - 주요 모델들
        1. **AutoModelForSequenceClassification**
           - 시퀀스(Text) 분류 작업을 위한 모델을 자동으로 로드한다.
           - 예: `AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")`
        2. **AutoModelForQuestionAnswering**
           - 질문-응답 작업을 위한 모델을 자동으로 로드한다.
           - 예: `AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased")`
        3. **AutoModelForTokenClassification**
           - 토큰 분류 작업(예: 개체명 인식)을 위한 모델을 자동으로 로드한다.
           - 예: `AutoModelForTokenClassification.from_pretrained("bert-base-uncased")`

In [1]:
from transformers import AutoModel, AutoTokenizer, AutoConfig

c:\Anaconda3\envs\dl\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model_id="bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_id)
print(type(tokenizer))
model = AutoModel.from_pretrained(model_id)
print(type(model))
config = AutoConfig.from_pretrained(model_id)
print(type(config))

<class 'transformers.models.bert.tokenization_bert_fast.BertTokenizerFast'>


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


<class 'transformers.models.bert.modeling_bert.BertModel'>
<class 'transformers.models.bert.configuration_bert.BertConfig'>


In [13]:
raw_txt = "I am a boy."
token = tokenizer(
    raw_txt,
    return_tensors="pt")
# 토큰화 결과: default-list, "pt"-torch.tensor, "tf"-tensorflow.tnesor, "np"-np.array
token
# input_ids: 토큰 id
# attention_mask: input_ids의 각 토큰들이 실제 토큰인지 [PAD] 토큰인지 여부
# 1: 유효(실제) 문자열 토큰, 0: [PAD]

{'input_ids': tensor([[ 101, 1045, 2572, 1037, 2879, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}

In [10]:
type(token['input_ids'])

list

In [14]:
# model
context_vector = model(**token)

In [15]:
context_vector

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.1779,  0.1550, -0.1707,  ...,  0.0410,  0.4759,  0.5032],
         [-0.1397,  0.2318, -0.2807,  ...,  0.0090,  0.5446,  0.2929],
         [-0.2585,  0.2294,  0.1758,  ..., -0.1466,  0.3572,  0.2033],
         ...,
         [-0.0560,  0.1688,  0.5574,  ..., -0.2375,  0.8589, -1.2152],
         [-0.1773, -0.7334, -0.3479,  ...,  0.4004,  0.6529, -0.3144],
         [ 0.7643,  0.1765, -0.0638,  ..., -0.0126, -0.4987, -0.3970]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-8.9368e-01, -4.8297e-01, -7.3119e-01,  8.0403e-01,  4.5552e-01,
         -1.2779e-01,  8.5415e-01,  1.8577e-01, -7.4326e-01, -9.9997e-01,
         -3.2631e-01,  9.5897e-01,  9.8007e-01,  5.2694e-01,  9.2719e-01,
         -7.5958e-01, -5.3485e-01, -6.0855e-01,  3.3701e-01, -6.5122e-01,
          7.5661e-01,  9.9989e-01, -6.7539e-02,  3.4064e-01,  4.4406e-01,
          9.8609e-01, -7.1002e-01,  8.9186e-01,  9.6556e-01,  7.603

In [ ]:
context_vector.last_hidden_state.shape
# [1: 문서(문장) 개수, 5:토큰 수, 768: embedding vector의 차원 수]

torch.Size([1, 7, 768])

In [25]:
# 토크나이저 
t = tokenizer.encode("I am a boy.") # 토큰 아이디들만 리턴
t

[101, 1045, 2572, 1037, 2879, 1012, 102]

In [26]:
tokenizer.decode(t)

'[CLS] i am a boy. [SEP]'

In [30]:
# 개별 토큰 확인
tokenizer.convert_tokens_to_ids("am")
tokenizer.convert_tokens_to_ids(["I", "am", "a", "boy"])

[100, 2572, 1037, 2879]

In [29]:
tokenizer.convert_ids_to_tokens(2572)

'am'

## kcbert
- BERT 모델을 한글 텍스트로 학습 시킨 Pretrained model.
    - BERT는 Transformer의 Encoder 부분을 이용해 구현된 언어모델
    - https://arxiv.org/abs/1810.04805 
- https://huggingface.co/beomi/kcbert-base

### 토크나이저 모델 load

In [31]:
from transformers import AutoModel, AutoTokenizer

model_id = "beomi/kcbert-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id)

In [32]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30000, 768, padding_idx=0)
    (position_embeddings): Embedding(300, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

### 입력값 토큰화

In [56]:
sentences = [
    "안녕"
    "Hugging Face는 인공지능(AI)과 자연어 처리(NLP) 분야에서 혁신적인 도구와 모델을 제공하는 AI 스타트업이다.",
    "2016년에 설립된 이 회사는 주로 오픈소스 라이브러리와 사전 학습된 NLP 모델을 제공을 제공한다."
]

In [57]:
token = tokenizer(sentences[0])
token

{'input_ids': [2, 19017, 5121, 4223, 4403, 4403, 18940, 39, 4243, 4773, 4226, 4008, 14583, 25061, 11, 22502, 12, 321, 10459, 4071, 9810, 11, 47, 4450, 4579, 12, 16029, 7971, 13064, 8097, 867, 4228, 4196, 16505, 4027, 13248, 7966, 22502, 12296, 4104, 22192, 4020, 17, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [58]:
token.keys()
# 기본적으로 input_ids, attention_mask는 준다
# input_ids: 토큰 id들
# attention_mask: input_ids의 개별 토큰들이 실제 값인지 [PAD]인지 구분. (1: 실제 토큰, 0: 패딩)
# 이 모델은 token_type_ids를 더 준 것
# token_type_ids: 입력이 두 개 문서로 구성된 경우 (question-answering: Question, Context)
#                 각 토큰이 몇 번째 문서인지 구분하기 위한 값이다.
# input_ids:        [q, q, q, q, c, c, c]
# token_type_ids:   [0, 0, 0, 0, 1, 1, 1]

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [59]:
token_list = tokenizer(
    sentences, # 토큰화할 문서들
    max_length=10, # padding/truncation의 기준 길이 지정
    padding=True, # 패딩 추가. default: batch에서 가장 긴 sequence에 맞춰서 패딩. 
                                        # max_length가 설정된 경우는 max_length에 맞춘다.
    truncation=True, # max_length 보다 개수가 많은 seq일 경우 max_length에 맞춰 뒤에 토큰을 잘라낸다.
    return_tensors="pt"
)

In [60]:
token_list["input_ids"]

tensor([[    2, 19017,  5121,  4223,  4403,  4403, 18940,    39,  4243,     3],
        [    2, 26182,  4113, 20684,  4130,  2451, 22088, 20002, 15999,     3]])

In [61]:
token_list["attention_mask"]

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [62]:
token_list["token_type_ids"]

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [52]:
token_list["input_ids"][0].shape

torch.Size([10])

truncation 된 거임. 10개로 잘라냄

### BERT 모델을 이용해 context vector 추출
#### Model 추론결과
- **last_hidden_state**
    - 모든 token들에 대한 feature
    - 출력이 **many**인 작업에 사용한다.
- **pooler_output**
    - 입력 문장, 텍스트에 대한 context vector 이다.
    - 이 값은 **문장을 입력받아 처리하는 task**(ex: 문서분류-감정분석,문장카테고리분류, 문장유사도 분석)의 입력으로 사용한다.

In [63]:
output = model(**token_list)

In [64]:
output.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [66]:
output["pooler_output"].shape

torch.Size([2, 768])